# Combined the following:
- https://docs.fast.ai/tutorial.data.html#Text
- https://docs.fast.ai/text.html#Building-a-classifier

<br>
<br>
<br>
<br>

# Notes:

Fastai's text module has three steps:
1. Pre-process data
2. Fine-tune a pre-trained model
3. Create other models (e.g. classifiers) on top of the encoder of the fine-tuned model

<br>
<br>
According to: https://gilberttanner.com/blog/fastai-sentiment-analysis

Standard FastAI training pipeline
1. Finding the best learning rate
2. Training the top layers
3. Unfreezing all layers

<br>

Regarding `fit_one_cycle`

- https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html
- https://sgugger.github.io/the-1cycle-policy.html
<br>
<br>

In [1]:
from fastai.text import *

In [2]:
imdb = untar_data(URLs.IMDB_SAMPLE)

In [3]:
imdb

PosixPath('/Users/p787144/.fastai/data/imdb_sample')

In [4]:
df = pd.read_csv(imdb/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


# Fine-tune

Can also be done by reading from dataframe, according to https://gilberttanner.com/blog/fastai-sentiment-analysis

`train_df, valid_df = df.loc[:12000,:], df.loc[12000:,:]`

`data_lm = TextLMDataBunch.from_df(path, train_df, valid_df, text_cols=10, bs=32)`

In [5]:
data_lm = (
    TextList
    .from_csv(imdb, 'texts.csv', cols='text')
    .split_by_rand_pct()
    .label_for_lm()
    .databunch()
)

data_lm.save()

data.show_batch( ) shows the beginning of each sequence of text along the batch dimension (the target being to guess the next word).

You  may notice that there are quite a few strange tokens starting with xx. These are special FastAI tokens that have the following meanings:

- xxunk: Token used instead of unknown words (words not found in the vocabulary).
- xxbos: Beginning of a text.
- xxfld: Represents separate parts of your document (several columns in a dataframe) like headline, body, summary, etc.
- xxmaj: Indicates that the next word starts with a capital, e.g. “House” will be tokenized as “xxmaj house”.
- xxup: Indicates that next word is written in all caps, e.g. “WHY” will be tokenized as “xxup why ”.
- xxrep: Token indicates that a character is repeated n times, e.g. if you have  10 $ in a row it will be tokenized as “xxrep 10 $” (in general “xxrep n  {char}”)
- xxwrep: Indicates that a word is repeated n times.
- xxpad : Token used as padding (so every text has the same length)

In [6]:
data_lm.show_batch()

idx,text
0,! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk ! xxbos xxmaj every once in a long while a movie will come along that will be so awful that i feel compelled to warn people . xxmaj if i labor all my days and i can save
1,". xxmaj the cover of the video box , if you can find the video , is extremely xxunk . i 'd xxunk that the two women on the cover are n't even in the film . \n \n xxmaj anyway , i was either born a decade too late to appreciate the xxunk points of this film or ... it is simply pointless junk . i 'm heavily"
2,xxunk - directed films and i enjoyed them more than his much more famous films which starred xxmaj louise xxmaj brooks ( such as xxup xxunk xxup 's xxup box ) . xxmaj it 's probably because both are very similar to the xxmaj neo - xxmaj xxunk films that the xxmaj xxunk xxunk in the 1940s and 50s . xxmaj this style film called for using non - actors
3,"hunter is then hired to find her , but along the way he has to brave the natives , lead by a man who calls himself "" xxmaj the xxmaj devil "" ( hence the title ) . xxmaj the film basically just plods along for xxunk five minutes and there really are n't many scenes of interest . xxmaj it 's a real shame that xxmaj xxunk xxmaj franco"
4,no big names in this film and the acting is very cheesy . xxmaj the perception of paintball is pretty bad too . xxmaj they seem to xxunk that everyone is going to xxunk and all this other crap . xxmaj it just was a lousy movie in my opinion and does n't give a real perception what paintball is . xxmaj to be honest real paintball is n't all


<br>

I believe in below it doesn't matter to specify 
`learn.unfreeze()` or not

In [7]:
learn = language_model_learner(data_lm, AWD_LSTM)
learn.fit_one_cycle(2, 1e-2)
learn.save('mini_train_lm')
learn.save_encoder('mini_train_encoder')

epoch,train_loss,valid_loss,accuracy,time
0,4.291222,3.774262,0.288795,03:53
1,4.009931,3.746215,0.292292,04:07


To evaluate your language model, you can run the Learner.predict method and specify the number of words you want it to guess.

`learn.predict("This is a review about", n_words=10)`

Or

In [8]:
learn.show_results()

text,target,pred
xxbos xxmaj this movie is a must - see movie for all . xxmaj xxunk should see this xxunk documentary,"from the point - of - view of the soldier , as should everyone in xxmaj america . xxmaj the",", the start of of - view . the xxunk who and well the in the xxunk . xxmaj the"
"xxunk as a xxunk lady xxunk on top of him and later his xxunk xxunk xxunk were hysterical , but",then i remembered it was n't supposed to be a comedy . i 'm xxunk xxunk my brain to find,the the was the as a a to be a xxunk . xxmaj was not to to xxunk . see
the lives of a multi - ethnic mix of not so ordinary people in the rural xxmaj xxunk xxmaj xxunk,". xxmaj solid directing and writing along with fine acting , especially the performances by xxmaj xxunk xxmaj xxunk and",", xxmaj the xxunk , xxunk , with a cinematography , and in xxunk of xxmaj xxunk xxmaj xxunk and"
"friday and xxup the xxup xxunk xxup story . xxmaj both of these movies features women with a strong ,","xxunk screen presence and who played independent , xxunk - feminist characters . xxmaj in both movies , both women","xxunk personality . . a is a women xxunk xxunk xxunk xxunk . xxmaj the the the , xxmaj are"
a single interesting or xxunk thing xxmaj james said during the course of the show . xxmaj he is xxup,"that boring and forgettable . xxmaj in fact , one of the xxunk flat out xxunk him he was n't","ok xxunk , xxunk . xxmaj the fact , he of the best of - of is in has in"


# Build a classifier

In [9]:
data_clas = (
    TextList
    .from_csv(imdb, 'texts.csv', cols='text', vocab=data_lm.vocab)
    .split_from_df(col='is_valid')
    .label_from_df(cols='label')
    .databunch(bs=32)
)

In [10]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the xxunk is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj xxunk",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk xxunk experience in general . xxmaj with graphics",positive


### Method 1:

Currently has AWD_LSTM, Transformer and TransformerXL

In [11]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('mini_train_encoder')
learn.fit_one_cycle(2, slice(1e-3,1e-2))
learn.save('mini_train_clas')

epoch,train_loss,valid_loss,accuracy,time
0,0.683442,0.653294,0.640000,05:39
1,0.627569,0.571250,0.705000,05:46


In [14]:
learn.show_results()

text,target,prediction
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive,positive
"xxbos xxmaj to review this movie , i without any doubt would have to quote that memorable scene in xxmaj tarantino 's "" xxmaj pulp xxmaj fiction "" ( xxunk ) when xxmaj jules and xxmaj vincent are talking about xxmaj xxunk xxmaj xxunk and what she does for a living . xxmaj jules tells xxmaj vincent that the "" xxmaj only thing she did worthwhile was pilot "" .",negative,negative
"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one",negative,negative
"xxbos xxmaj the trouble with the book , "" xxmaj memoirs of a xxmaj geisha "" is that it had xxmaj japanese xxunk but underneath the xxunk it was all an xxmaj american man 's way of thinking . xxmaj reading the book is like watching a magnificent ballet with great music , sets , and costumes yet performed by xxunk animals dressed in those xxunk far from xxmaj japanese",negative,negative
"xxbos xxmaj bonanza had a great cast of wonderful actors . xxmaj xxunk xxmaj xxunk , xxmaj pernell xxmaj whitaker , xxmaj michael xxmaj xxunk , xxmaj dan xxmaj blocker , and even xxmaj guy xxmaj williams ( as the cousin who was brought in for several episodes during 1964 to replace xxmaj adam when he was leaving the series ) . xxmaj the cast had chemistry , and they",positive,positive


### Method 2:

In [12]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('mini_train_encoder')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.645538,0.647899,0.615000,05:46


Unfreeze the model and fine-tune it.

In [14]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.553105,0.554503,0.695000,05:57


In [15]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.475513,0.479004,0.790000,06:05


In [16]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.3629, 0.6371]))